## Jorge Pablo Ávila Gómez

# Ejercicio 3 (análisis de sentimiento): (3 puntos)
Se  pide  desarrollar  un  notebook  de  Jupyter,  denominado  “sentiment.ipynb”,  en  el  que  se  utilice como fuente de datos Kafka, y en concreto el topic kafkaTwitter. Para cada nuevo tweet que llegue al sistema, se analizará su polaridad dividiendo el tweet por palabras y realizando la suma de la polaridad de todas las palabras que aparezcan en los ficheros “positive_lex.txt” y “negative_lex.txt”. Las palabras del lexicon negativo tienen un signo menos delante, por lo  que  restarán  a  la  puntuación  final  del  tweet.  Si  ninguna  de  las  palabras  del  tweet  está  almacenada en los ficheros proporcionados (o si las puntuaciones de las palabras positivas y negativas se anulan), la polaridad del tweet será 0.

La salida del sistema ha de imprimirse cada segundo, y consistirá en una línea por cada tweet recibido,  que  contenga  el  texto  del  propio  tweet,  a  continuación  la  puntuación  de  su  polaridad, y la palabra “NEUTRO” si la puntuación es igual 0, “POSITIVO” si es mayor que 0, y “NEGATIVO” si es menor que 0.

In [1]:
import findspark

# Se indica la ruta de spark:
findspark.init("C:\\Users\\JorgeAvila\\Documents\\spark-2.4.7-bin-hadoop2.7")

import os

os.environ[
    "PYSPARK_SUBMIT_ARGS"
] = "--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.7 pyspark-shell"

In [2]:
# Se importan los diferentes paquetes necesarios:
import pyspark
import pyspark.streaming
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

### Preparación del entorno Spark Streaming

In [3]:
sc = SparkContext("local[*]")
# Se inicializa SparkContext con la opción "local[*]" para que use todos los núcleos del equipo
ssc = StreamingContext(sc, batchDuration=1)
# Se crea el StreamingContext indicando que la duración del batch es 5s como pide el enunciado.

### Coneixón con kafka e introducción de los datos

In [4]:
# Para crear el stream de datos utilizamos la siguiente función:
tweetsDS = KafkaUtils.createDirectStream(
    ssc,  # StreamingContext con el que conectar kafka
    topics=["kafkaTwitter"],  # El topic con el que queremos conectar.
    kafkaParams={
        "bootstrap.servers": "192.168.1.100:9092, 192.168.1.100:9093"
    },  # Las direcciones de los brokers.
)

Las puntuaciones de las palabras positivas y negativas de los archivos "positive_lex.txt" y "negative_lex.txt" se almacenan en un diccionario de python (Words). Siendo la llave la palabra y el valor la puntuación.

In [5]:
# Diccionario con las palabras que tiene puntuación.
Words = {}
# Words['palabra'] = puntuación
f = open("positive_lex.txt", "r", encoding="utf8")
for line in f:
    Words[line.strip().split(" ")[0]] = float(line.strip().split(" ")[1])
f.close()

f = open("negative_lex.txt", "r", encoding="utf8")
for line in f:
    Words[line.strip().split(" ")[0]] = float(line.strip().split(" ")[1])
f.close()

# Función que calcula la puntuación un tweet
def calculate_punt(tweet):
    punt = 0
    words = tweet[1]
    for word in words:
        if word in Words:
            punt += Words[word]
    if punt < 0:
        sentiment = "NEGATIVO"
    elif punt > 0:
        sentiment = "POSITIVO"
    else:
        sentiment = "NEUTRO"
    return tweet[0], punt, sentiment
    # return (texto del tweet, puntuación, positivo, negativo o neutro)

Se ha preparado también la función "calculate_punt", que recibe un tweet y calcula la puntuación de cada palabra mirando el diccionario Words, sumando la puntuación total de todas las palabras. La salida está en el formato que se pide para el ejercicio, devuelve el texto del tweet, la puntuación total y el sentimiento, si es positivo, negativo o neutro.

El tratamiento que se le da al stream de datos es muy sencillo. Primero, usamos una función map para seleccinar el texto del tweet que se encuentra en la posición 1 de la tuple que nos llega de kafka. Después, aplicamos de nuevo una función map a cada tweet que nos devuelve una tupla con el texto del tweet en la primera posición y, en la segunda posición una lista de las palabras que forman el tweet. Por último, aplicamos una función map con la función calculate_punt descrita anteriormente que nos devuelve ya cada tweet con su puntuación correspondiente.

In [6]:
# Extraemos el texto del tweet que se encuentra en la posición 1 de una tupla.
tweets = tweetsDS.map(lambda tweet: tweet[1])
# Dividimos cada tweet en las palabra que lo constituyen.
tweet_plus_words = tweets.map(lambda tweet: (tweet, tweet.strip().split(" ")))
# Usamos la función calculate_punt para calcular la puntuación de cada tweet.
tweet_plus_words.map(calculate_punt).pprint()

### Ejecución del programa

In [7]:
ssc.start()

-------------------------------------------
Time: 2021-01-10 11:49:48
-------------------------------------------
('Mañana, viernes 6M, Seminario Álgebra y Combinatoria en Fac. #Ciencias #UAM Organiza: @_ICMAT http://t.co/iYdShSNAhC http://t.co/qsN05667uH', 0, 'NEUTRO')

-------------------------------------------
Time: 2021-01-10 11:49:49
-------------------------------------------
('"Hace 2 años Rajoy fue a explicar el caso Bárcenas y tardó mucho en hacerlo y podemos decir que fueron insuficiente" @ainhat #ReuniónRatoM4', 0, 'NEUTRO')

-------------------------------------------
Time: 2021-01-10 11:49:50
-------------------------------------------
('La Oreja de Van Gogh-Jueves con letra https://t.co/H3tj3QzxS2 vía @YouTube ..Hermosa y triste historia a la vez :´(', -0.259, 'NEGATIVO')

-------------------------------------------
Time: 2021-01-10 11:49:51
-------------------------------------------
('"@marianorajoy es quien está liderando el proceso de renovación y cambio del PP" @mar

-------------------------------------------
Time: 2021-01-10 11:50:24
-------------------------------------------
('Declaraciones de bienes y programas electorales municipales http://t.co/sciIg2LRfC #RafaPresidente @RafaGTovar #digoSIvotoPSOE', 0, 'NEUTRO')

-------------------------------------------
Time: 2021-01-10 11:50:25
-------------------------------------------
('¡La conferencia de @SamsungMobile empieza ya! Hay liveblog: http://t.co/iE63Amvfiu y video http://t.co/81dSJi426q http://t.co/8xfoq9Rlk8', 0, 'NEUTRO')

-------------------------------------------
Time: 2021-01-10 11:50:26
-------------------------------------------
('“Fotografías de #voluntarios en proyectos en India” es una de las muestras programadas en #XISemanaSolidaridad #UAM http://t.co/tykxIcVPem', 0, 'NEUTRO')

-------------------------------------------
Time: 2021-01-10 11:50:27
-------------------------------------------
('.@davtun2 Los beneficios de salir a correr http://t.co/MzHyXBgqKO http://t.co/bA7wK3I

In [8]:
ssc.stop(stopSparkContext=True, stopGraceFully=True)

-------------------------------------------
Time: 2021-01-10 11:50:50
-------------------------------------------
('DENTRO DE UNOS MINUTOS NUEVO VÍDEO!! :D', 0, 'NEUTRO')

-------------------------------------------
Time: 2021-01-10 11:50:51
-------------------------------------------
('Hoy me voy a dormir tarde , ya q no tengo colee :D', 0.375, 'POSITIVO')

-------------------------------------------
Time: 2021-01-10 11:50:52
-------------------------------------------
('La habilidad de Mariano Rajoy para captar talento no deja de sorprenderme… https://t.co/MpN21aC6R9', 1.262, 'POSITIVO')

-------------------------------------------
Time: 2021-01-10 11:50:53
-------------------------------------------
('Ey! ♛      BJ     ♛  muchas gracias por el follow! :D', 0, 'NEUTRO')

-------------------------------------------
Time: 2021-01-10 11:50:54
-------------------------------------------
('Nos alegra que nos reconozcan como herramienta de cambio pero hemos de ser serios con el capital pol

-------------------------------------------
Time: 2021-01-10 11:51:26
-------------------------------------------
('Doctor, ¿por qué me mareo al levantarme del sofá? http://t.co/5SC1uWeR5W http://t.co/581OqRsLYX', 0, 'NEUTRO')

-------------------------------------------
Time: 2021-01-10 11:51:27
-------------------------------------------
('#chiste ¿Crees que tengo la nariz grande? —Pues tienes una nariz común. —¿En serio? —Sí, común tucán. :D xD', 0, 'NEUTRO')

-------------------------------------------
Time: 2021-01-10 11:51:28
-------------------------------------------
('Este PC personalizado de Minecraft es perfecto para los fans del juego http://t.co/vSH3zf7bhF', 0.875, 'POSITIVO')

-------------------------------------------
Time: 2021-01-10 11:51:29
-------------------------------------------
('El nuevo iPhone 6 de cerca (¡en vídeo!) - http://t.co/RBcYme04ix http://t.co/h1GEltORcI', 0, 'NEUTRO')

-------------------------------------------
Time: 2021-01-10 11:51:30
----------

-------------------------------------------
Time: 2021-01-10 11:52:02
-------------------------------------------
('@leonardo_ji Cierto es pero juego en PC y pasaron de la plataforma,ha sido jugar y 20 tios con Aimbot...', 0, 'NEUTRO')

-------------------------------------------
Time: 2021-01-10 11:52:03
-------------------------------------------
('El reloj de Google se ha actualizado... ¿para mal o para bien? http://t.co/l69JNDZmgb http://t.co/XDnMmGSxir', -0.375, 'NEGATIVO')

-------------------------------------------
Time: 2021-01-10 11:52:04
-------------------------------------------
('@OMAHALOUIS en mi casa no se hace pure :p', 0.375, 'POSITIVO')

-------------------------------------------
Time: 2021-01-10 11:52:05
-------------------------------------------
('@ElPutoHank @bigotix__ @mpreyb @SherlockBond_ estás quedando fatal con todos a la vez :D', -0.025000000000000022, 'NEGATIVO')

-------------------------------------------
Time: 2021-01-10 11:52:06
----------------------

-------------------------------------------
Time: 2021-01-10 11:52:38
-------------------------------------------
('Tas Loco Cero Cabida Ah La Gilada :P', 0, 'NEUTRO')

-------------------------------------------
Time: 2021-01-10 11:52:39
-------------------------------------------
('Muchas gracias amiga @Say_Linette te quiero mucho!!! #SerasMiDama :D', 0, 'NEUTRO')

-------------------------------------------
Time: 2021-01-10 11:52:40
-------------------------------------------
('Increible las jugadas que se nos pierden, Dodgers lo tenia hecho en el 8vo pero bota el juego de forma ridicula y nos tumba el Parley y....', 0.313, 'POSITIVO')

-------------------------------------------
Time: 2021-01-10 11:52:41
-------------------------------------------
('Ninguna medida que Rajoy anunció contra la corrupción está en vigor http://t.co/uHk5MGYsdZ http://t.co/jEHtHJyimw', 0, 'NEUTRO')

-------------------------------------------
Time: 2021-01-10 11:52:42
------------------------------------

Primero indicar que se ha realizado la modificación sugerida en el enunciado y tweet_producer.py envía 1 tweet por segundo.

En los resultados vemos que la salida se estabiliza a 1 tweet por segundo, como sería de esperar por el ritmo de producción. También vemos que se ejecuta correctamente una salida por segundo.

Por otro lado, la salida del programa es una tupla por cada tweet. En la tupla se incluye el texto del tweet, la puntuación y finalmente, si se considera neutro, positivo o negativo. Por tanto, podemos concluir que el programa se ejecuta correctamente obteniendose los resultados esperados.